# Проект для «Викишоп» с BERT


Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

План работы: 
1. Загрузка и подготовка данных.
2. Обучение разных моделей. 
3. Вывод.

Описание данных:
 - *text*  - текст комментария, 
 - *toxic* - целевой признак.

### Содержание

<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#1.-Подготовка" data-toc-modified-id="1.-Подготовка-1.2">1. Подготовка<a id="preparing"></a></a></span><ul class="toc-item"><li><span><a href="#1.1-Открытие-и-изучиние-файлов" data-toc-modified-id="1.1-Открытие-и-изучиние-файлов-1.2.1">1.1 Открытие и изучиние файлов<a id="open"></a></a></span></li><li><span><a href="#1.2-Подготовка-данных-к-обучению" data-toc-modified-id="1.2-Подготовка-данных-к-обучению-1.2.2">1.2 Подготовка данных к обучению<a id="preparation"></a></a></span><ul class="toc-item"><li><span><a href="#Очистка-данных" data-toc-modified-id="Очистка-данных-1.2.2.1">Очистка данных</a></span></li><li><span><a href="#Разбиение-датасета-на-тестовые,-тренировочные-и-валидационные-выборки." data-toc-modified-id="Разбиение-датасета-на-тестовые,-тренировочные-и-валидационные-выборки.-1.2.2.2">Разбиение датасета на тестовые, тренировочные и валидационные выборки.<a id="valid"></a></a></span></li><li><span><a href="#Вычисление-TF-IDF" data-toc-modified-id="Вычисление-TF-IDF-1.2.2.3">Вычисление TF-IDF</a></span></li></ul></li></ul></li><li><span><a href="#2.-Обучение-" data-toc-modified-id="2.-Обучение--1.3">2. Обучение <a id="training"></a></a></span><ul class="toc-item"><li><span><a href="#2.1-Логистическая-регрессия" data-toc-modified-id="2.1-Логистическая-регрессия-1.3.1">2.1 Логистическая регрессия<a id="3.1"></a></a></span></li><li><span><a href="#2.2-Случайный-лес" data-toc-modified-id="2.2-Случайный-лес-1.3.2">2.2 Случайный лес<a id="3.3"></a></a></span></li><li><span><a href="#2.3-Дерево-решений" data-toc-modified-id="2.3-Дерево-решений-1.3.3">2.3 Дерево решений<a id="3.4"></a></a></span></li><li><span><a href="#2.4-LGBMClassifier" data-toc-modified-id="2.4-LGBMClassifier-1.3.4">2.4 LGBMClassifier</a></span></li></ul></li><li><span><a href="#3.-Тестирование" data-toc-modified-id="3.-Тестирование-1.4">3. Тестирование</a></span><ul class="toc-item"><li><span><a href="#3.1-Логистическая-регрессия" data-toc-modified-id="3.1-Логистическая-регрессия-1.4.1">3.1 Логистическая регрессия</a></span></li><li><span><a href="#3.2-LGBMClassifier" data-toc-modified-id="3.2-LGBMClassifier-1.4.2">3.2 LGBMClassifier</a></span></li></ul></li><li><span><a href="#Вывод-" data-toc-modified-id="Вывод--1.5">Вывод <a id="conclusion"></a></a></li></ul></li></ul></div>

## 1. Подготовка<a id="preparing"></a>
### 1.1 Открытие и изучиние файлов<a id="open"></a>

In [1]:
# Импорт библеотек

from IPython.display import display
import pandas as pd
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import fbeta_score, make_scorer
from tqdm import notebook
from sklearn.tree import DecisionTreeClassifier
import re
from lightgbm import LGBMClassifier

In [2]:
# Открытие данных
df = pd.read_csv('toxic_comments.csv')
display(df.head(5))

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
df.info()
df.duplicated().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


0

В загруженном датасете 159571 записей. Пропусков нет, дубликатов нет.

### 1.2 Подготовка данных к обучению<a id="preparation"></a>
#### Очистка данных

In [4]:
def clear_text(text):
    return " ".join(re.sub(r'[^a-zA-Z]',' ', text).split())
df['lemma'] = df['text'].apply(clear_text)

In [5]:
print(df['lemma'].sample(5))

23324     Backstreet Boys Backstreet Boys article was bl...
141663    Image Why can we use these images Image X japa...
88428     Hey Jerk You need to read the editnotice well ...
35802     YA GODDAMIT IDIOT MY IP CHANGES WHEN AN ADMIST...
130301    Thank you for experimenting with Wikipedia You...
Name: lemma, dtype: object


#### Разбиение датасета на тестовые, тренировочные и валидационные выборки.<a id='valid'></a>

In [6]:
x_train, x_test, y_train, y_test = train_test_split(
    df.lemma, df.toxic, test_size=0.4, random_state=12345)
x_valid, x_test, y_valid, y_test = train_test_split(
    x_test, y_test, test_size=0.5, random_state=12345)

print('Размер тренировочной выборки:', x_train.count())
print('Размер валидационной выборки:',x_valid.count())
print('Размер тестовой выборки:',x_test.count())


Размер тренировочной выборки: 95742
Размер валидационной выборки: 31914
Размер тестовой выборки: 31915


#### Вычисление TF-IDF
Для тренировочной выборки

In [7]:
corpus = x_train
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords, lowercase=True)
tf_idf = count_tf_idf.fit_transform(corpus)
tf_idf.shape

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sugan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(95742, 125610)

Для валидационной выборки

In [8]:
corpus = x_valid
tf_idf_valid = count_tf_idf.transform(corpus)
tf_idf_valid.shape

(31914, 125610)

Для тестовой выборки

In [9]:
corpus = x_test
tf_idf_test = count_tf_idf.transform(corpus)
tf_idf_test.shape

(31915, 125610)

## 2. Обучение <a id='training'></a>
### 2.1 Логистическая регрессия<a id="3.1"></a>

In [10]:
model = LogisticRegression(random_state=12345, class_weight='balanced').fit(tf_idf, y_train)
predictions = model.predict(tf_idf_valid)
f1 = f1_score(y_valid, predictions)
print('f1 для логистической регрессии без кросс-валидации:', f1)

# Список показателей f1 моделей
f1_list = []
f1_list.append(f1.round(2))

f1 для логистической регрессии без кросс-валидации: 0.7532967032967033


D:\Program Files\anaconda3\envs\praktikum\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


### 2.2 Случайный лес<a id="3.3"></a>

In [11]:
best_model = None
best_f1 = 0


for depth in notebook.tqdm(range(50, 150, 10)):
    model = RandomForestClassifier(max_depth=depth, random_state=12345, n_estimators=1).fit(tf_idf, y_train)
    preditions = model.predict(tf_idf_valid)
    f1 = f1_score(y_valid,preditions)
    
    if best_f1 < f1:
        best_model = model
        best_f1 = f1
        best_depth = depth
        
            
print('Налилучший показатель F1 = {} для случайного леса достигается при глубине равной {}:'.format(best_f1,best_depth) )

# запись f1
f1_list.append(best_f1.round(2))

  0%|          | 0/10 [00:00<?, ?it/s]

Налилучший показатель F1 = 0.45306532663316584 для случайного леса достигается при глубине равной 130:


### 2.3 Дерево решений<a id="3.4"></a>

In [12]:
best_depth = 0
best_model = None
best_f1 = 0

for depth in notebook.tqdm(range(50, 150, 10)):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth).fit(tf_idf, y_train)
    predictions = model.predict(tf_idf_valid)
    f1 = f1_score(y_valid, predictions)
    if best_f1 < f1:
        best_f1 = f1
        best_depth = depth
        best_model = model

print('Налилучший показатель F1 = {} для дерева решений достигается при глубине равной {}:'.format(best_f1,best_depth) )

# запись f1
f1_list.append(best_f1.round(2))

  0%|          | 0/10 [00:00<?, ?it/s]

Налилучший показатель F1 = 0.7340479836651354 для дерева решений достигается при глубине равной 100:


### 2.4 LGBMClassifier

In [13]:
best_model = None
best_f1 = 0
best_est = 0
for est in notebook.tqdm(range(100, 151, 25)):
    model = LGBMClassifier(random_state=12345, n_estimators=est).fit(tf_idf, y_train)
    preditions = model.predict(tf_idf_valid)
    f1 = f1_score(y_valid, preditions)
    if f1 > best_f1:
        best_model = model
        best_f1 = f1
        best_est = est
            
        
print('Налилучший показатель F1 = {} для дерева решений достигается при количестве деревьев равным {}:'.format(best_f1,best_est) )
# запись f1
f1_list.append(best_f1.round(2))

  0%|          | 0/3 [00:00<?, ?it/s]

Налилучший показатель F1 = 0.763264586638463 для дерева решений достигается при количестве деревьев равным 150:


In [14]:
index_list = ['Логистическая регрессия без кросс-валидации', 'Дерево решений', "Случайный лес", 'LGBMClassifier' ]
d = {'f1': f1_list}
result = pd.DataFrame(data=d, index = index_list)
display(result)

,f1
Логистическая регрессия без кросс-валидации,0.75
Дерево решений,0.45
Случайный лес,0.73
LGBMClassifier,0.76


Для тестирования возьмем Логистическую регрессию и LGBMClassifier, так как показатели F1 больше 0.75.

## 3. Тестирование
### 3.1 Логистическая регрессия

In [15]:
model = LogisticRegression(random_state=12345, class_weight='balanced').fit(tf_idf, y_train)
predictions = model.predict(tf_idf_test)
f1 = f1_score(y_test, predictions)
print('f1 для логистической регрессии:', f1)

f1 для логистической регрессии: 0.7524587893059981


D:\Program Files\anaconda3\envs\praktikum\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


### 3.2 LGBMClassifier

In [16]:
model = LGBMClassifier(random_state=12345, n_estimators=500).fit(tf_idf, y_train)
preditions = model.predict(tf_idf_test)
f1 = f1_score(y_test, preditions)
print('f1 для LGBMClassifier:', f1)

f1 для LGBMClassifier: 0.7743170349747696


## Вывод <a id='conclusion'></a>

Проведена предобработка данных и обучена модель линейной регрессией, случайным лесом, деревом решиний.
Наилучшие показатели метрики показала модель, обученная LGBMClassifier: F1 = 0.77.

Исходя из  результатов теста предлагаю модель LGBMClassifier, так как удовлетворяет условию поставленной задаче.
